In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from PIL import Image

train_transform = transforms.Compose([
    transforms.Resize(384, interpolation=Image.BICUBIC),
    transforms.CenterCrop(384),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize(384, interpolation=Image.BICUBIC),
    transforms.CenterCrop(384),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('data/splited/train', transform=train_transform)
test_dataset = datasets.ImageFolder('data/test', transform=test_transform)

batch_size = 16
num_workers = 2
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
model = models.regnet_y_32gf()
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load('training/best.pt'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.eval()
predictions = []
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, dim=1)
        predictions.extend(predicted.cpu().numpy())

file_names = [os.path.splitext(os.path.basename(path))[0]
                for path, _ in test_dataset.imgs]
class_labels = [os.path.basename(class_path)
                for class_path in train_dataset.class_to_idx.keys()]
predicted_labels = [class_labels[prediction] for prediction in predictions]

df = pd.DataFrame({'id': file_names, 'label': predicted_labels})
df.to_csv('predictions.csv', index=False)